In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Dropout, Concatenate, LSTM, Embedding, Bidirectional, GRU, SpatialDropout1D, Conv1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.initializers import Constant
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, classification_report, f1_score
from sklearn.utils.class_weight import compute_class_weight

import transformers
from transformers import AutoTokenizer, AutoModel, TFAutoModel

import torch
import matplotlib.pyplot as plt

2024-07-14 05:12:08.276755: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-14 05:12:08.276876: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-14 05:12:08.413673: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
!sudo apt-get update
!sudo apt-get install openjdk-11-jdk-headless -qq > /dev/null

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1581 B]
Hit:2 https://packages.cloud.google.com/apt gcsfuse-focal InRelease            
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]      
Hit:4 https://packages.cloud.google.com/apt cloud-sdk InRelease                
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [1556 kB]
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease                         
Hit:7 https://packages.cloud.google.com/apt google-fast-socket InRelease       
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]        
Get:9 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [3745 kB]
Hit:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease 
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [3895 kB]
Get:12 http://security.ubuntu.com/ubuntu focal-security/main amd64 Pac

In [4]:
!git clone https://github.com/hhdang241/kltn

Cloning into 'kltn'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 104 (delta 45), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (104/104), 3.12 MiB | 3.62 MiB/s, done.
Resolving deltas: 100% (45/45), done.


# Accelerator

In [5]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0': raise SystemError('GPU device not found')
print('Found GPU at:', device_name)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

Found GPU at: /device:GPU:0


'Tesla P100-PCIE-16GB'

# Import & Preprocess datasets

In [6]:
TRAIN_PATH = 'kltn/Data/Sách/book_reviews_train.csv'
VAL_PATH = 'kltn/Data/Sách/book_reviews_val.csv'
TEST_PATH = 'kltn/Data/Sách/book_reviews_test.csv'
MAX_LENGTH = 128
BATCH_SIZE = 32

In [7]:
df_train = pd.read_csv(TRAIN_PATH, encoding='utf8')
df_val = pd.read_csv(VAL_PATH, encoding='utf8')
df_test = pd.read_csv(TEST_PATH, encoding='utf8')

In [8]:
df_train.head(10)

,review,content_related,author,quality_and_appearance,manufacture_and_distribution,packaging_and_delivery,price_and_preference,customer_service,overall_experience
0,là tuổi_thơ Năm nay ra tập kỷ_niệm nên mình nổ...,0,0,0,0,0,1,0,0
1,Đóng_gói hàng rất cẩn bọc sách rất đẹp,0,0,1,0,1,0,0,0
2,rất đáng tiền,0,0,0,0,0,1,0,0
3,giao hang nhanh san pham dung nhu mo ta,0,0,0,0,1,0,0,1
4,Đóng_gói kỹ sách đẹp dù bị cấn gáy và trầy 1 t...,1,0,1,0,1,0,0,0
5,Mình đặt 2 cuốn hãng luật và Xứ_Cát vào buổi s...,0,0,1,0,1,0,0,1
6,Sách không còn gì để bàn_cãi rồi giao hàng đỉn...,0,0,1,0,1,0,0,1
7,Sách to và dày hơn mình nghĩ Dịch_thuật chất_l...,1,1,3,0,0,1,0,0
8,Sách in tốt Bìa đẹp Tuy là bìa mềm mà khổ sách...,1,0,1,0,0,0,0,0
9,Nội_dung sách hay đọc rất cuốn và buồn để lại ...,1,0,2,0,3,0,0,0


# Get labels

In [9]:
def get_label_aspect(df):
    
    label_aspect = df.iloc[:, 1:].values

    # Replace -1 with 0 and other values with 1
    label_aspect[label_aspect == -1] = 0
    label_aspect[label_aspect != 0] = 1

    return label_aspect


def get_label_sentiment(df):
    
    content_related = df.content_related
    author = df.author
    quality_and_appearance = df.quality_and_appearance
    manufacture_and_distribution = df.manufacture_and_distribution
    packaging_and_delivery = df.packaging_and_delivery
    price_and_preference = df.price_and_preference
    customer_service = df.customer_service
    overall_experience = df.overall_experience

    sentiment_dict = {
        'content_related': content_related, 
        'author': author,
        'quality_and_appearance': quality_and_appearance,
        'manufacture_and_distribution': manufacture_and_distribution,
        'packaging_and_delivery': packaging_and_delivery,
        'price_and_preference': price_and_preference,
        'customer_service': customer_service, 
        'overall_experience': overall_experience
    }
    
    df_sentiment = pd.DataFrame(sentiment_dict)

    return df_sentiment


def get_aspect_sentiment_label(label_dict, num_classes):
    content_related = to_categorical(label_dict['content_related'], num_classes=num_classes)
    author = to_categorical(label_dict['author'], num_classes=num_classes)
    quality_and_appearance = to_categorical(label_dict['quality_and_appearance'], num_classes=num_classes)
    manufacture_and_distribution = to_categorical(label_dict['manufacture_and_distribution'], num_classes=num_classes)
    packaging_and_delivery = to_categorical(label_dict['packaging_and_delivery'], num_classes=num_classes)
    price_and_preference = to_categorical(label_dict['price_and_preference'], num_classes=num_classes)
    customer_service = to_categorical(label_dict['customer_service'], num_classes=num_classes)
    overall_experience = to_categorical(label_dict['overall_experience'], num_classes=num_classes)
    
    return (
        content_related,
        author,
        quality_and_appearance,
        manufacture_and_distribution,
        packaging_and_delivery,
        price_and_preference,
        customer_service,
        overall_experience
    )

In [10]:
label_aspect_train = get_label_aspect(df_train)
label_aspect_val = get_label_aspect(df_val)
label_aspect_test = get_label_aspect(df_test)

In [11]:
label_sentiment_train = get_label_sentiment(df_train)
label_sentiment_val = get_label_sentiment(df_val)
label_sentiment_test = get_label_sentiment(df_test)

In [12]:
label_aspect_train.shape

(5415, 8)

In [13]:
label_sentiment_train.shape

(5415, 8)

In [14]:
content_related_train, author_train, quality_and_appearance_train, manufacture_and_distribution_train, packaging_and_delivery_train, price_and_preference_train, customer_service_train, overall_experience_train = get_aspect_sentiment_label(label_sentiment_train, num_classes=4)

content_related_val, author_val, quality_and_appearance_val, manufacture_and_distribution_val, packaging_and_delivery_val, price_and_preference_val, customer_service_val, overall_experience_val = get_aspect_sentiment_label(label_sentiment_val, num_classes=4)

content_related_test, author_test, quality_and_appearance_test, manufacture_and_distribution_test, packaging_and_delivery_test, price_and_preference_test, customer_service_test, overall_experience_test = get_aspect_sentiment_label(label_sentiment_test, num_classes=4)

# Create word embedding matrix

In [15]:
cmt_train = df_train['review'].astype('str').to_list()
cmt_val = df_val['review'].astype('str').to_list()
cmt_test = df_test['review'].astype('str').to_list()

In [16]:
#tok = Tokenizer(filters='')
#tok.fit_on_texts(cmt_train)
#tok.fit_on_texts(cmt_val)
#tok.fit_on_texts(cmt_test)

#tokenized_train = tok.texts_to_sequences(cmt_train)
#tokenized_val = tok.texts_to_sequences(cmt_val)
#tokenized_test = tok.texts_to_sequences(cmt_test)

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-large", use_fast=False)

#padded_train = pad_sequences(tokenized_train, padding='post', maxlen=MAX_LENGTH)
#padded_val = pad_sequences(tokenized_val, padding='post', maxlen=MAX_LENGTH)
#padded_test = pad_sequences(tokenized_test, padding='post', maxlen=MAX_LENGTH)

padded_train = tokenizer(cmt_train, padding='max_length', truncation=True, max_length=MAX_LENGTH, return_tensors="np")['input_ids']
padded_val = tokenizer(cmt_val, padding='max_length', truncation=True, max_length=MAX_LENGTH, return_tensors="np")['input_ids']
padded_test = tokenizer(cmt_test, padding='max_length', truncation=True, max_length=MAX_LENGTH, return_tensors="np")['input_ids']

config.json:   0%|          | 0.00/558 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

In [17]:
padded_train

array([[    0,     8,  6765, ...,     1,     1,     1],
       [    0, 55662,  1685, ...,     1,     1,     1],
       [    0,    59,   463, ...,     1,     1,     1],
       ...,
       [    0,  5947,    45, ...,     1,     1,     1],
       [    0,  6042,     7, ...,     1,     1,     1],
       [    0,  6042,     7, ...,     1,     1,     1]])

In [18]:
phobert = AutoModel.from_pretrained("vinai/phobert-large", output_hidden_states=True)

pytorch_model.bin:   0%|          | 0.00/1.48G [00:00<?, ?B/s]

In [19]:
#tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-large", use_fast=False)

# Get the vocabulary and the size of the vocabulary
vocabulary = tokenizer.get_vocab()
vocab_size = len(vocabulary)

# Get the embedding dimension
embedding_dim = phobert.embeddings.word_embeddings.embedding_dim

# Extract the embedding weights directly and convert to NumPy array
embedding_matrix = phobert.embeddings.word_embeddings.weight.detach().cpu().numpy()

In [20]:
embedding_matrix.shape

(64001, 1024)

# Model building

## BiLSTM

In [21]:
with device:

    # Number of labels
    num_label = 8

    # Input layer
    inputs = Input(shape=(MAX_LENGTH,))

    # Embedding layer
    embed = Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        embeddings_initializer=Constant(embedding_matrix),
        trainable=True
    )(inputs)
    
    lstm = Bidirectional(LSTM(units = 200, activation = 'tanh'))(embed)
    dense2 = Dense(units = 128, activation = 'relu')(lstm)
    dropout1 = Dropout(rate = 0.2)(dense2)
    dense3 = Dense(units = 64, activation = 'relu')(dropout1)
    dense4 = Dense(units = 32, activation = 'relu')(dense3)

    # Output layers
    out_aspect = Dense(units=num_label, activation='sigmoid', name='out_aspect')(dense4)

    # Helper function to concatenate aspect and dense4 output
    def create_output_layer(aspect_index, name):
        aspect_slice = out_aspect[:, aspect_index:aspect_index + 1]
        concatenated = tf.keras.layers.Concatenate(axis=1)([aspect_slice, dense4])
        output_layer = Dense(units=4, activation='softmax', name=name)(concatenated)
        return output_layer

    out_content_related = create_output_layer(0, 'out_content_related')
    out_author = create_output_layer(1, 'out_author')
    out_quality_and_appearance = create_output_layer(2, 'out_quality_and_appearance')
    out_manufacture_and_distribution = create_output_layer(3, 'out_manufacture_and_distribution')
    out_packaging_and_delivery = create_output_layer(4, 'out_packaging_and_delivery')
    out_price_and_preference = create_output_layer(5, 'out_price_and_preference')
    out_customer_service = create_output_layer(6, 'out_customer_service')
    out_overall_experience = create_output_layer(7, 'out_overall_experience')

    # Define the model
    model = Model(
        inputs=inputs,
        outputs=[
            out_aspect, out_content_related, out_author, out_quality_and_appearance,
            out_manufacture_and_distribution, out_packaging_and_delivery,
            out_price_and_preference, out_customer_service, out_overall_experience
        ]
    )
    
    losses = {
        "out_aspect": "binary_crossentropy",
        "out_content_related": "categorical_crossentropy",
        "out_author": "categorical_crossentropy",
        "out_quality_and_appearance": "categorical_crossentropy",
        "out_manufacture_and_distribution": "categorical_crossentropy",
        "out_packaging_and_delivery": "categorical_crossentropy",
        "out_price_and_preference": "categorical_crossentropy",
        "out_customer_service": "categorical_crossentropy",
        "out_overall_experience": "categorical_crossentropy"
    }
 
    metrics={
        "out_aspect": 'accuracy',
        "out_content_related": 'accuracy',
        "out_author": 'accuracy',
        "out_quality_and_appearance": 'accuracy',
        "out_manufacture_and_distribution": 'accuracy',
        "out_packaging_and_delivery": 'accuracy',
        "out_price_and_preference": 'accuracy',
        "out_customer_service": 'accuracy',
        "out_overall_experience": 'accuracy'
    }
    
    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss=losses,
        metrics=metrics
    )

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 128, 1024) │ 65,537,024 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 400)       │  1,960,000 │ embedding[0][0]   │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │     51,328 │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 128)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 64)        │      8,256 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 32)        │      2,080 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ out_aspect (Dense)  │ (None, 8)         │        264 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item (GetItem)  │ (None, 1)         │          0 │ out_aspect[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_1          │ (None, 1)         │          0 │ out_aspect[0][0]  │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_2          │ (None, 1)         │          0 │ out_aspect[0][0]  │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_3          │ (None, 1)         │          0 │ out_aspect[0][0]  │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_4          │ (None, 1)         │          0 │ out_aspect[0][0]  │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_5          │ (None, 1)         │          0 │ out_aspect[0][0]  │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_6          │ (None, 1)         │          0 │ out_aspect[0][0]  │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_7          │ (None, 1)         │          0 │ out_aspect[0][0]  │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 33)        │          0 │ get_item[0][0],   │
│ (Concatenate)       │                   │            │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 33)        │          0 │ get_item_1[0][0], │
│ (Concatenate)       │                   │            │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 33)        │          0 │ get_item_2[0][0]

 Total params: 67,560,040 (257.72 MB)

 Trainable params: 67,560,040 (257.72 MB)

 Non-trainable params: 0 (0.00 B)

In [22]:
# Fit model
history = model.fit(
    x=padded_train,
    y={
        "out_aspect": label_aspect_train,
        "out_content_related": content_related_train,
        "out_author": author_train,
        "out_quality_and_appearance": quality_and_appearance_train,
        "out_manufacture_and_distribution": manufacture_and_distribution_train,
        "out_packaging_and_delivery": packaging_and_delivery_train,
        "out_price_and_preference": price_and_preference_train,
        "out_customer_service": customer_service_train,
        "out_overall_experience": overall_experience_train
    },
    validation_data=(
        padded_val,
        {
            "out_aspect": label_aspect_val,
            "out_content_related": content_related_val,
            "out_author": author_val,
            "out_quality_and_appearance": quality_and_appearance_val,
            "out_manufacture_and_distribution": manufacture_and_distribution_val,
            "out_packaging_and_delivery": packaging_and_delivery_val,
            "out_price_and_preference": price_and_preference_val,
            "out_customer_service": customer_service_val,
            "out_overall_experience": overall_experience_val
        }
    ),
    batch_size=BATCH_SIZE,
    epochs=100,
    callbacks=EarlyStopping(
        monitor='val_loss', 
        patience=3,
        min_delta=0.001,
        mode='min',
        verbose=1
    ),
    verbose=1
)

Epoch 1/100
170/170 ━━━━━━━━━━━━━━━━━━━━ 20s 60ms/step - loss: 9.4957 - out_aspect_accuracy: 0.0484 - out_author_accuracy: 0.9513 - out_content_related_accuracy: 0.6663 - out_customer_service_accuracy: 0.7102 - out_manufacture_and_distribution_accuracy: 0.5031 - out_overall_experience_accuracy: 0.1850 - out_packaging_and_delivery_accuracy: 0.4673 - out_price_and_preference_accuracy: 0.4897 - out_quality_and_appearance_accuracy: 0.3379 - val_loss: 5.9861 - val_out_aspect_accuracy: 0.4133 - val_out_author_accuracy: 0.9523 - val_out_content_related_accuracy: 0.6806 - val_out_customer_service_accuracy: 0.9725 - val_out_manufacture_and_distribution_accuracy: 0.9957 - val_out_overall_experience_accuracy: 0.5694 - val_out_packaging_and_delivery_accuracy: 0.4986 - val_out_price_and_preference_accuracy: 0.9162 - val_out_quality_and_appearance_accuracy: 0.3237
Epoch 2/100
170/170 ━━━━━━━━━━━━━━━━━━━━ 9s 55ms/step - loss: 5.9846 - out_aspect_accuracy: 0.4000 - out_author_accuracy: 0.9528 - out_co

### Evaluation

In [23]:
pred_test = model.predict(padded_test)

22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step


In [24]:
pred_aspect_test = round(pd.DataFrame(pred_test[0]), 0)
aspect_true = label_aspect_test.flatten()
aspect_predict = pred_aspect_test.values.flatten()

weighted_f1_aspect = f1_score(aspect_true, aspect_predict, average='weighted')
print(f'Weighted F1 Score for aspect detection: {weighted_f1_aspect}')

Weighted F1 Score for aspect detection: 0.8869102248874658


In [25]:
pred_content_related_test = np.where(np.argmax(pred_test[1], axis=1)==3, -1, np.argmax(pred_test[1], axis=1))
pred_author_test = np.where(np.argmax(pred_test[2], axis=1)==3, -1, np.argmax(pred_test[2], axis=1))
pred_quality_and_appearance_test = np.where(np.argmax(pred_test[3], axis=1)==3, -1, np.argmax(pred_test[3], axis=1))
pred_manufacture_and_distribution_test = np.where(np.argmax(pred_test[4], axis=1)==3, -1, np.argmax(pred_test[4], axis=1))
pred_packaging_and_delivery_test = np.where(np.argmax(pred_test[5], axis=1)==3, -1, np.argmax(pred_test[5], axis=1))
pred_price_and_preference_test = np.where(np.argmax(pred_test[6], axis=1)==3, -1, np.argmax(pred_test[6], axis=1))
pred_customer_service_test = np.where(np.argmax(pred_test[7], axis=1)==3, -1, np.argmax(pred_test[7], axis=1))
pred_overall_experience_test = np.where(np.argmax(pred_test[8], axis=1)==3, -1, np.argmax(pred_test[8], axis=1))

predict_test = pd.DataFrame({
    'content_related': pred_content_related_test,
    'author': pred_author_test,
    'quality_and_appearance': pred_quality_and_appearance_test,
    'manufacture_and_distribution': pred_manufacture_and_distribution_test,
    'packaging_and_delivery': pred_packaging_and_delivery_test,
    'price_and_preference': pred_price_and_preference_test,
    'customer_service': pred_customer_service_test,
    'overall_experience': pred_overall_experience_test
})

sentiment_true = df_test.iloc[:, 1:].values.flatten()
sentiment_predict = predict_test.values.flatten()

weighted_f1_sentiment = f1_score(sentiment_true, sentiment_predict, average='weighted')
print(f'Weighted F1 Score for sentiment classification: {weighted_f1_sentiment}')

Weighted F1 Score for sentiment classification: 0.8670248142649625


In [26]:
del model

## BiLSTM + Conv1D

In [27]:
with device:

    # Number of labels
    num_label = 8

    # Input layer
    inputs = Input(shape=(MAX_LENGTH,))

    # Embedding layer
    embed = Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        embeddings_initializer=Constant(embedding_matrix),
        trainable=True
    )(inputs)
    
    dropout1 = SpatialDropout1D(0.2)(embed)

    lstm = Bidirectional(LSTM(units = 200, activation = 'tanh', return_sequences = True))(dropout1)
    conv = Conv1D(128, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(lstm)

    avg_pool1 = GlobalAveragePooling1D()(conv)
    max_pool1 = GlobalMaxPooling1D()(conv)


    concat = Concatenate(axis=-1)([avg_pool1, max_pool1])

    dense2 = Dense(units = 128, activation = 'relu')(concat)
    dropout1 = Dropout(rate = 0.2)(dense2)
    dense3 = Dense(units = 64, activation = 'relu')(dropout1)
    dense4 = Dense(units = 32, activation = 'relu')(dense3)
    
    # Output layers
    out_aspect = Dense(units=num_label, activation='sigmoid', name='out_aspect')(dense4)

    # Helper function to concatenate aspect and dense4 output
    def create_output_layer(aspect_index, name):
        aspect_slice = out_aspect[:, aspect_index:aspect_index + 1]
        concatenated = tf.keras.layers.Concatenate(axis=1)([aspect_slice, dense4])
        output_layer = Dense(units=4, activation='softmax', name=name)(concatenated)
        return output_layer

    out_content_related = create_output_layer(0, 'out_content_related')
    out_author = create_output_layer(1, 'out_author')
    out_quality_and_appearance = create_output_layer(2, 'out_quality_and_appearance')
    out_manufacture_and_distribution = create_output_layer(3, 'out_manufacture_and_distribution')
    out_packaging_and_delivery = create_output_layer(4, 'out_packaging_and_delivery')
    out_price_and_preference = create_output_layer(5, 'out_price_and_preference')
    out_customer_service = create_output_layer(6, 'out_customer_service')
    out_overall_experience = create_output_layer(7, 'out_overall_experience')

    # Define the model
    model = Model(
        inputs=inputs,
        outputs=[
            out_aspect, out_content_related, out_author, out_quality_and_appearance,
            out_manufacture_and_distribution, out_packaging_and_delivery,
            out_price_and_preference, out_customer_service, out_overall_experience
        ]
    )
    
    losses = {
        "out_aspect": "binary_crossentropy",
        "out_content_related": "categorical_crossentropy",
        "out_author": "categorical_crossentropy",
        "out_quality_and_appearance": "categorical_crossentropy",
        "out_manufacture_and_distribution": "categorical_crossentropy",
        "out_packaging_and_delivery": "categorical_crossentropy",
        "out_price_and_preference": "categorical_crossentropy",
        "out_customer_service": "categorical_crossentropy",
        "out_overall_experience": "categorical_crossentropy"
    }
 
    metrics={
        "out_aspect": 'accuracy',
        "out_content_related": 'accuracy',
        "out_author": 'accuracy',
        "out_quality_and_appearance": 'accuracy',
        "out_manufacture_and_distribution": 'accuracy',
        "out_packaging_and_delivery": 'accuracy',
        "out_price_and_preference": 'accuracy',
        "out_customer_service": 'accuracy',
        "out_overall_experience": 'accuracy'
    }
    
    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss=losses,
        metrics=metrics
    )

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 128, 1024) │ 65,537,024 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spatial_dropout1d   │ (None, 128, 1024) │          0 │ embedding_1[0][0] │
│ (SpatialDropout1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_1     │ (None, 128, 400)  │  1,960,000 │ spatial_dropout1… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 127, 128)  │    102,528 │ bidirectional_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 128)       │          0 │ conv1d[0][0]      │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ conv1d[0][0]      │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_8       │ (None, 256)       │          0 │ global_average_p… │
│ (Concatenate)       │                   │            │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 128)       │     32,896 │ concatenate_8[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 128)       │          0 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 64)        │      8,256 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 32)        │      2,080 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ out_aspect (Dense)  │ (None, 8)         │        264 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_8          │ (None, 1)         │          0 │ out_aspect[0][0]  │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_9          │ (None, 1)         │          0 │ out_aspect[0][0]  │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_10         │ (None, 1)         │          0 │ out_aspect[0][0]  │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_11         │ (None, 1)         │          0 │ out_aspect[0][0]  │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_12         │ (None, 1)         │          0 │ out_aspect[0][0]  │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_13         │ (None, 1)         │          0 │ out_aspect[0][0]

 Total params: 67,644,136 (258.04 MB)

 Trainable params: 67,644,136 (258.04 MB)

 Non-trainable params: 0 (0.00 B)

In [28]:
# Fit model
history = model.fit(
    x=padded_train,
    y={
        "out_aspect": label_aspect_train,
        "out_content_related": content_related_train,
        "out_author": author_train,
        "out_quality_and_appearance": quality_and_appearance_train,
        "out_manufacture_and_distribution": manufacture_and_distribution_train,
        "out_packaging_and_delivery": packaging_and_delivery_train,
        "out_price_and_preference": price_and_preference_train,
        "out_customer_service": customer_service_train,
        "out_overall_experience": overall_experience_train
    },
    validation_data=(
        padded_val,
        {
            "out_aspect": label_aspect_val,
            "out_content_related": content_related_val,
            "out_author": author_val,
            "out_quality_and_appearance": quality_and_appearance_val,
            "out_manufacture_and_distribution": manufacture_and_distribution_val,
            "out_packaging_and_delivery": packaging_and_delivery_val,
            "out_price_and_preference": price_and_preference_val,
            "out_customer_service": customer_service_val,
            "out_overall_experience": overall_experience_val
        }
    ),
    batch_size=BATCH_SIZE,
    epochs=100,
    callbacks=EarlyStopping(
        monitor='val_loss', 
        patience=3,
        min_delta=0.001,
        mode='min',
        verbose=1
    ),
    verbose=1
)

Epoch 1/100
170/170 ━━━━━━━━━━━━━━━━━━━━ 19s 66ms/step - loss: 9.8028 - out_aspect_accuracy: 0.0665 - out_author_accuracy: 0.4886 - out_content_related_accuracy: 0.5483 - out_customer_service_accuracy: 0.8361 - out_manufacture_and_distribution_accuracy: 0.5288 - out_overall_experience_accuracy: 0.2934 - out_packaging_and_delivery_accuracy: 0.4148 - out_price_and_preference_accuracy: 0.5882 - out_quality_and_appearance_accuracy: 0.2842 - val_loss: 5.7160 - val_out_aspect_accuracy: 0.0000e+00 - val_out_author_accuracy: 0.9523 - val_out_content_related_accuracy: 0.6806 - val_out_customer_service_accuracy: 0.9725 - val_out_manufacture_and_distribution_accuracy: 0.9957 - val_out_overall_experience_accuracy: 0.7355 - val_out_packaging_and_delivery_accuracy: 0.4884 - val_out_price_and_preference_accuracy: 0.9162 - val_out_quality_and_appearance_accuracy: 0.4321
Epoch 2/100
170/170 ━━━━━━━━━━━━━━━━━━━━ 10s 57ms/step - loss: 5.9006 - out_aspect_accuracy: 0.0137 - out_author_accuracy: 0.9528 - o

### Evaluation

In [29]:
pred_test = model.predict(padded_test)

22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step


In [30]:
pred_aspect_test = round(pd.DataFrame(pred_test[0]), 0)
aspect_true = label_aspect_test.flatten()
aspect_predict = pred_aspect_test.values.flatten()

weighted_f1_aspect = f1_score(aspect_true, aspect_predict, average='weighted')
print(f'Weighted F1 Score for aspect detection: {weighted_f1_aspect}')

Weighted F1 Score for aspect detection: 0.8778199264019297


In [31]:
pred_content_related_test = np.where(np.argmax(pred_test[1], axis=1)==3, -1, np.argmax(pred_test[1], axis=1))
pred_author_test = np.where(np.argmax(pred_test[2], axis=1)==3, -1, np.argmax(pred_test[2], axis=1))
pred_quality_and_appearance_test = np.where(np.argmax(pred_test[3], axis=1)==3, -1, np.argmax(pred_test[3], axis=1))
pred_manufacture_and_distribution_test = np.where(np.argmax(pred_test[4], axis=1)==3, -1, np.argmax(pred_test[4], axis=1))
pred_packaging_and_delivery_test = np.where(np.argmax(pred_test[5], axis=1)==3, -1, np.argmax(pred_test[5], axis=1))
pred_price_and_preference_test = np.where(np.argmax(pred_test[6], axis=1)==3, -1, np.argmax(pred_test[6], axis=1))
pred_customer_service_test = np.where(np.argmax(pred_test[7], axis=1)==3, -1, np.argmax(pred_test[7], axis=1))
pred_overall_experience_test = np.where(np.argmax(pred_test[8], axis=1)==3, -1, np.argmax(pred_test[8], axis=1))

predict_test = pd.DataFrame({
    'content_related': pred_content_related_test,
    'author': pred_author_test,
    'quality_and_appearance': pred_quality_and_appearance_test,
    'manufacture_and_distribution': pred_manufacture_and_distribution_test,
    'packaging_and_delivery': pred_packaging_and_delivery_test,
    'price_and_preference': pred_price_and_preference_test,
    'customer_service': pred_customer_service_test,
    'overall_experience': pred_overall_experience_test
})

sentiment_true = df_test.iloc[:, 1:].values.flatten()
sentiment_predict = predict_test.values.flatten()

weighted_f1_sentiment = f1_score(sentiment_true, sentiment_predict, average='weighted')
print(f'Weighted F1 Score for sentiment classification: {weighted_f1_sentiment}')

Weighted F1 Score for sentiment classification: 0.8646717812031753


In [32]:
del model

## BiGRU

In [33]:
with device:

    # Number of labels
    num_label = 8

    # Input layer
    inputs = Input(shape=(MAX_LENGTH,))

    # Embedding layer
    embed = Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        embeddings_initializer=Constant(embedding_matrix),
        trainable=True
    )(inputs)
    
    lstm = Bidirectional(GRU(units = 200, activation = 'tanh'))(embed)
    dense2 = Dense(units = 128, activation = 'relu')(lstm)
    dropout1 = Dropout(rate = 0.2)(dense2)
    dense3 = Dense(units = 64, activation = 'relu')(dropout1)
    dense4 = Dense(units = 32, activation = 'relu')(dense3)
    
    # Output layers
    out_aspect = Dense(units=num_label, activation='sigmoid', name='out_aspect')(dense4)

    # Helper function to concatenate aspect and dense4 output
    def create_output_layer(aspect_index, name):
        aspect_slice = out_aspect[:, aspect_index:aspect_index + 1]
        concatenated = tf.keras.layers.Concatenate(axis=1)([aspect_slice, dense4])
        output_layer = Dense(units=4, activation='softmax', name=name)(concatenated)
        return output_layer

    out_content_related = create_output_layer(0, 'out_content_related')
    out_author = create_output_layer(1, 'out_author')
    out_quality_and_appearance = create_output_layer(2, 'out_quality_and_appearance')
    out_manufacture_and_distribution = create_output_layer(3, 'out_manufacture_and_distribution')
    out_packaging_and_delivery = create_output_layer(4, 'out_packaging_and_delivery')
    out_price_and_preference = create_output_layer(5, 'out_price_and_preference')
    out_customer_service = create_output_layer(6, 'out_customer_service')
    out_overall_experience = create_output_layer(7, 'out_overall_experience')

    # Define the model
    model = Model(
        inputs=inputs,
        outputs=[
            out_aspect, out_content_related, out_author, out_quality_and_appearance,
            out_manufacture_and_distribution, out_packaging_and_delivery,
            out_price_and_preference, out_customer_service, out_overall_experience
        ]
    )
    
    losses = {
        "out_aspect": "binary_crossentropy",
        "out_content_related": "categorical_crossentropy",
        "out_author": "categorical_crossentropy",
        "out_quality_and_appearance": "categorical_crossentropy",
        "out_manufacture_and_distribution": "categorical_crossentropy",
        "out_packaging_and_delivery": "categorical_crossentropy",
        "out_price_and_preference": "categorical_crossentropy",
        "out_customer_service": "categorical_crossentropy",
        "out_overall_experience": "categorical_crossentropy"
    }
 
    metrics={
        "out_aspect": 'accuracy',
        "out_content_related": 'accuracy',
        "out_author": 'accuracy',
        "out_quality_and_appearance": 'accuracy',
        "out_manufacture_and_distribution": 'accuracy',
        "out_packaging_and_delivery": 'accuracy',
        "out_price_and_preference": 'accuracy',
        "out_customer_service": 'accuracy',
        "out_overall_experience": 'accuracy'
    }
    
    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss=losses,
        metrics=metrics
    )

model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 128, 1024) │ 65,537,024 │ input_layer_2[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_2     │ (None, 400)       │  1,471,200 │ embedding_2[0][0] │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 128)       │     51,328 │ bidirectional_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 128)       │          0 │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 64)        │      8,256 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 32)        │      2,080 │ dense_7[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ out_aspect (Dense)  │ (None, 8)         │        264 │ dense_8[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_16         │ (None, 1)         │          0 │ out_aspect[0][0]  │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_17         │ (None, 1)         │          0 │ out_aspect[0][0]  │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_18         │ (None, 1)         │          0 │ out_aspect[0][0]  │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_19         │ (None, 1)         │          0 │ out_aspect[0][0]  │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_20         │ (None, 1)         │          0 │ out_aspect[0][0]  │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_21         │ (None, 1)         │          0 │ out_aspect[0][0]  │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_22         │ (None, 1)         │          0 │ out_aspect[0][0]  │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_23         │ (None, 1)         │          0 │ out_aspect[0][0]  │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_17      │ (None, 33)        │          0 │ get_item_16[0][0… │
│ (Concatenate)       │                   │            │ dense_8[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_18      │ (None, 33)        │          0 │ get_item_17[0][0… │
│ (Concatenate)       │                   │            │ dense_8[0][0]     │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 67,071,240 (255.86 MB)

 Trainable params: 67,071,240 (255.86 MB)

 Non-trainable params: 0 (0.00 B)

In [34]:
# Fit model
history = model.fit(
    x=padded_train,
    y={
        "out_aspect": label_aspect_train,
        "out_content_related": content_related_train,
        "out_author": author_train,
        "out_quality_and_appearance": quality_and_appearance_train,
        "out_manufacture_and_distribution": manufacture_and_distribution_train,
        "out_packaging_and_delivery": packaging_and_delivery_train,
        "out_price_and_preference": price_and_preference_train,
        "out_customer_service": customer_service_train,
        "out_overall_experience": overall_experience_train
    },
    validation_data=(
        padded_val,
        {
            "out_aspect": label_aspect_val,
            "out_content_related": content_related_val,
            "out_author": author_val,
            "out_quality_and_appearance": quality_and_appearance_val,
            "out_manufacture_and_distribution": manufacture_and_distribution_val,
            "out_packaging_and_delivery": packaging_and_delivery_val,
            "out_price_and_preference": price_and_preference_val,
            "out_customer_service": customer_service_val,
            "out_overall_experience": overall_experience_val
        }
    ),
    batch_size=BATCH_SIZE,
    epochs=100,
    callbacks=EarlyStopping(
        monitor='val_loss', 
        patience=3,
        min_delta=0.001,
        mode='min',
        verbose=1
    ),
    verbose=1
)

Epoch 1/100
170/170 ━━━━━━━━━━━━━━━━━━━━ 16s 59ms/step - loss: 10.5126 - out_aspect_accuracy: 0.0969 - out_author_accuracy: 0.6910 - out_content_related_accuracy: 0.2734 - out_customer_service_accuracy: 0.8815 - out_manufacture_and_distribution_accuracy: 0.0898 - out_overall_experience_accuracy: 0.4553 - out_packaging_and_delivery_accuracy: 0.4576 - out_price_and_preference_accuracy: 0.5822 - out_quality_and_appearance_accuracy: 0.2475 - val_loss: 5.5713 - val_out_aspect_accuracy: 0.3194 - val_out_author_accuracy: 0.9523 - val_out_content_related_accuracy: 0.6806 - val_out_customer_service_accuracy: 0.9725 - val_out_manufacture_and_distribution_accuracy: 0.9957 - val_out_overall_experience_accuracy: 0.7355 - val_out_packaging_and_delivery_accuracy: 0.4884 - val_out_price_and_preference_accuracy: 0.9162 - val_out_quality_and_appearance_accuracy: 0.4321
Epoch 2/100
170/170 ━━━━━━━━━━━━━━━━━━━━ 9s 54ms/step - loss: 5.6447 - out_aspect_accuracy: 0.3319 - out_author_accuracy: 0.9584 - out_c

### Evaluation

In [35]:
pred_test = model.predict(padded_test)

22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step


In [36]:
pred_aspect_test = round(pd.DataFrame(pred_test[0]), 0)
aspect_true = label_aspect_test.flatten()
aspect_predict = pred_aspect_test.values.flatten()

weighted_f1_aspect = f1_score(aspect_true, aspect_predict, average='weighted')
print(f'Weighted F1 Score for aspect detection: {weighted_f1_aspect}')

Weighted F1 Score for aspect detection: 0.8646809756505044


In [37]:
pred_content_related_test = np.where(np.argmax(pred_test[1], axis=1)==3, -1, np.argmax(pred_test[1], axis=1))
pred_author_test = np.where(np.argmax(pred_test[2], axis=1)==3, -1, np.argmax(pred_test[2], axis=1))
pred_quality_and_appearance_test = np.where(np.argmax(pred_test[3], axis=1)==3, -1, np.argmax(pred_test[3], axis=1))
pred_manufacture_and_distribution_test = np.where(np.argmax(pred_test[4], axis=1)==3, -1, np.argmax(pred_test[4], axis=1))
pred_packaging_and_delivery_test = np.where(np.argmax(pred_test[5], axis=1)==3, -1, np.argmax(pred_test[5], axis=1))
pred_price_and_preference_test = np.where(np.argmax(pred_test[6], axis=1)==3, -1, np.argmax(pred_test[6], axis=1))
pred_customer_service_test = np.where(np.argmax(pred_test[7], axis=1)==3, -1, np.argmax(pred_test[7], axis=1))
pred_overall_experience_test = np.where(np.argmax(pred_test[8], axis=1)==3, -1, np.argmax(pred_test[8], axis=1))

predict_test = pd.DataFrame({
    'content_related': pred_content_related_test,
    'author': pred_author_test,
    'quality_and_appearance': pred_quality_and_appearance_test,
    'manufacture_and_distribution': pred_manufacture_and_distribution_test,
    'packaging_and_delivery': pred_packaging_and_delivery_test,
    'price_and_preference': pred_price_and_preference_test,
    'customer_service': pred_customer_service_test,
    'overall_experience': pred_overall_experience_test
})

sentiment_true = df_test.iloc[:, 1:].values.flatten()
sentiment_predict = predict_test.values.flatten()

weighted_f1_sentiment = f1_score(sentiment_true, sentiment_predict, average='weighted')
print(f'Weighted F1 Score for sentiment classification: {weighted_f1_sentiment}')

Weighted F1 Score for sentiment classification: 0.8512215527270572


In [38]:
del model

## BiGRU + Conv1D

In [39]:
class ApplyAttentionMask(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(ApplyAttentionMask, self).__init__(**kwargs)

    def call(self, inputs):
        embed, attention_mask = inputs
        # Expand attention_mask to match the embedding dimensions
        attention_mask = tf.expand_dims(tf.cast(attention_mask, tf.float32), -1)
        # Apply attention mask to embeddings
        masked_embed = tf.multiply(embed, attention_mask)
        return masked_embed

In [40]:
with device:

    # Number of labels
    num_label = 8

    # Input layer
    inputs = Input(shape=(MAX_LENGTH,))

    # Embedding layer
    embed = Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        embeddings_initializer=Constant(embedding_matrix),
        trainable=True
    )(inputs)
    
    dropout1 = SpatialDropout1D(0.2)(embed)

    lstm = Bidirectional(GRU(units = 200, activation = 'tanh', return_sequences = True))(dropout1)
    conv = Conv1D(128, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(lstm)

    avg_pool1 = GlobalAveragePooling1D()(conv)
    max_pool1 = GlobalMaxPooling1D()(conv)


    concat = Concatenate(axis=-1)([avg_pool1, max_pool1])

    dense2 = Dense(units = 128, activation = 'relu')(concat)
    dropout1 = Dropout(rate = 0.2)(dense2)
    dense3 = Dense(units = 64, activation = 'relu')(dropout1)
    dense4 = Dense(units = 32, activation = 'relu')(dense3)
    
    # Output layers
    out_aspect = Dense(units=num_label, activation='sigmoid', name='out_aspect')(dense4)

    # Helper function to concatenate aspect and dense4 output
    def create_output_layer(aspect_index, name):
        aspect_slice = out_aspect[:, aspect_index:aspect_index + 1]
        concatenated = tf.keras.layers.Concatenate(axis=1)([aspect_slice, dense4])
        output_layer = Dense(units=4, activation='softmax', name=name)(concatenated)
        return output_layer

    out_content_related = create_output_layer(0, 'out_content_related')
    out_author = create_output_layer(1, 'out_author')
    out_quality_and_appearance = create_output_layer(2, 'out_quality_and_appearance')
    out_manufacture_and_distribution = create_output_layer(3, 'out_manufacture_and_distribution')
    out_packaging_and_delivery = create_output_layer(4, 'out_packaging_and_delivery')
    out_price_and_preference = create_output_layer(5, 'out_price_and_preference')
    out_customer_service = create_output_layer(6, 'out_customer_service')
    out_overall_experience = create_output_layer(7, 'out_overall_experience')

    # Define the model
    model = Model(
        inputs=inputs,
        outputs=[
            out_aspect, out_content_related, out_author, out_quality_and_appearance,
            out_manufacture_and_distribution, out_packaging_and_delivery,
            out_price_and_preference, out_customer_service, out_overall_experience
        ]
    )
    
    losses = {
        "out_aspect": "binary_crossentropy",
        "out_content_related": "categorical_crossentropy",
        "out_author": "categorical_crossentropy",
        "out_quality_and_appearance": "categorical_crossentropy",
        "out_manufacture_and_distribution": "categorical_crossentropy",
        "out_packaging_and_delivery": "categorical_crossentropy",
        "out_price_and_preference": "categorical_crossentropy",
        "out_customer_service": "categorical_crossentropy",
        "out_overall_experience": "categorical_crossentropy"
    }
 
    metrics={
        "out_aspect": 'accuracy',
        "out_content_related": 'accuracy',
        "out_author": 'accuracy',
        "out_quality_and_appearance": 'accuracy',
        "out_manufacture_and_distribution": 'accuracy',
        "out_packaging_and_delivery": 'accuracy',
        "out_price_and_preference": 'accuracy',
        "out_customer_service": 'accuracy',
        "out_overall_experience": 'accuracy'
    }
    
    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss=losses,
        metrics=metrics
    )

model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, 128, 1024) │ 65,537,024 │ input_layer_3[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spatial_dropout1d_1 │ (None, 128, 1024) │          0 │ embedding_3[0][0] │
│ (SpatialDropout1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_3     │ (None, 128, 400)  │  1,471,200 │ spatial_dropout1… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 127, 128)  │    102,528 │ bidirectional_3[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 128)       │          0 │ conv1d_1[0][0]    │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ conv1d_1[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_25      │ (None, 256)       │          0 │ global_average_p… │
│ (Concatenate)       │                   │            │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 128)       │     32,896 │ concatenate_25[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 128)       │          0 │ dense_9[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 64)        │      8,256 │ dropout_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 32)        │      2,080 │ dense_10[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ out_aspect (Dense)  │ (None, 8)         │        264 │ dense_11[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_24         │ (None, 1)         │          0 │ out_aspect[0][0]  │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_25         │ (None, 1)         │          0 │ out_aspect[0][0]  │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_26         │ (None, 1)         │          0 │ out_aspect[0][0]  │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_27         │ (None, 1)         │          0 │ out_aspect[0][0]  │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_28         │ (None, 1)         │          0 │ out_aspect[0][0]  │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_29         │ (None, 1)         │          0 │ out_aspect[0][0]

 Total params: 67,155,336 (256.18 MB)

 Trainable params: 67,155,336 (256.18 MB)

 Non-trainable params: 0 (0.00 B)

In [42]:
# Fit model
history = model.fit(
    x=padded_train,
    y={
        "out_aspect": label_aspect_train,
        "out_content_related": content_related_train,
        "out_author": author_train,
        "out_quality_and_appearance": quality_and_appearance_train,
        "out_manufacture_and_distribution": manufacture_and_distribution_train,
        "out_packaging_and_delivery": packaging_and_delivery_train,
        "out_price_and_preference": price_and_preference_train,
        "out_customer_service": customer_service_train,
        "out_overall_experience": overall_experience_train
    },
    validation_data=(
        padded_val,
        {
            "out_aspect": label_aspect_val,
            "out_content_related": content_related_val,
            "out_author": author_val,
            "out_quality_and_appearance": quality_and_appearance_val,
            "out_manufacture_and_distribution": manufacture_and_distribution_val,
            "out_packaging_and_delivery": packaging_and_delivery_val,
            "out_price_and_preference": price_and_preference_val,
            "out_customer_service": customer_service_val,
            "out_overall_experience": overall_experience_val
        }
    ),
    batch_size=BATCH_SIZE,
    epochs=100,
    callbacks=EarlyStopping(
        monitor='val_loss', 
        patience=3,
        min_delta=0.001,
        mode='min',
        verbose=1
    ),
    verbose=1
)

Epoch 1/100
170/170 ━━━━━━━━━━━━━━━━━━━━ 17s 61ms/step - loss: 9.9533 - out_aspect_accuracy: 0.3056 - out_author_accuracy: 0.5229 - out_content_related_accuracy: 0.2472 - out_customer_service_accuracy: 0.4659 - out_manufacture_and_distribution_accuracy: 0.5535 - out_overall_experience_accuracy: 0.6476 - out_packaging_and_delivery_accuracy: 0.2716 - out_price_and_preference_accuracy: 0.4651 - out_quality_and_appearance_accuracy: 0.3951 - val_loss: 5.7462 - val_out_aspect_accuracy: 0.4133 - val_out_author_accuracy: 0.9523 - val_out_content_related_accuracy: 0.6806 - val_out_customer_service_accuracy: 0.9725 - val_out_manufacture_and_distribution_accuracy: 0.9957 - val_out_overall_experience_accuracy: 0.7355 - val_out_packaging_and_delivery_accuracy: 0.4884 - val_out_price_and_preference_accuracy: 0.9162 - val_out_quality_and_appearance_accuracy: 0.4321
Epoch 2/100
170/170 ━━━━━━━━━━━━━━━━━━━━ 9s 55ms/step - loss: 5.9314 - out_aspect_accuracy: 0.4252 - out_author_accuracy: 0.9545 - out_co

### Evaluation

In [43]:
pred_test = model.predict(padded_test)

22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step


In [44]:
pred_aspect_test = round(pd.DataFrame(pred_test[0]), 0)
aspect_true = label_aspect_test.flatten()
aspect_predict = pred_aspect_test.values.flatten()

weighted_f1_aspect = f1_score(aspect_true, aspect_predict, average='weighted')
print(f'Weighted F1 Score for aspect detection: {weighted_f1_aspect}')

Weighted F1 Score for aspect detection: 0.8958719185060834


In [45]:
pred_content_related_test = np.where(np.argmax(pred_test[1], axis=1)==3, -1, np.argmax(pred_test[1], axis=1))
pred_author_test = np.where(np.argmax(pred_test[2], axis=1)==3, -1, np.argmax(pred_test[2], axis=1))
pred_quality_and_appearance_test = np.where(np.argmax(pred_test[3], axis=1)==3, -1, np.argmax(pred_test[3], axis=1))
pred_manufacture_and_distribution_test = np.where(np.argmax(pred_test[4], axis=1)==3, -1, np.argmax(pred_test[4], axis=1))
pred_packaging_and_delivery_test = np.where(np.argmax(pred_test[5], axis=1)==3, -1, np.argmax(pred_test[5], axis=1))
pred_price_and_preference_test = np.where(np.argmax(pred_test[6], axis=1)==3, -1, np.argmax(pred_test[6], axis=1))
pred_customer_service_test = np.where(np.argmax(pred_test[7], axis=1)==3, -1, np.argmax(pred_test[7], axis=1))
pred_overall_experience_test = np.where(np.argmax(pred_test[8], axis=1)==3, -1, np.argmax(pred_test[8], axis=1))

predict_test = pd.DataFrame({
    'content_related': pred_content_related_test,
    'author': pred_author_test,
    'quality_and_appearance': pred_quality_and_appearance_test,
    'manufacture_and_distribution': pred_manufacture_and_distribution_test,
    'packaging_and_delivery': pred_packaging_and_delivery_test,
    'price_and_preference': pred_price_and_preference_test,
    'customer_service': pred_customer_service_test,
    'overall_experience': pred_overall_experience_test
})

sentiment_true = df_test.iloc[:, 1:].values.flatten()
sentiment_predict = predict_test.values.flatten()

weighted_f1_sentiment = f1_score(sentiment_true, sentiment_predict, average='weighted')
print(f'Weighted F1 Score for sentiment classification: {weighted_f1_sentiment}')

Weighted F1 Score for sentiment classification: 0.877344441482388


In [46]:
df_true = pd.DataFrame(label_aspect_test, columns=['content_related', 'author', 'quality_and_appearance', 'manufacture_and_distribution', 'packaging_and_delivery', 'price_and_preference', 'customer_service', 'overall_experience'])
df_pred = pd.DataFrame(pred_aspect_test.values, columns=['content_related', 'author', 'quality_and_appearance', 'manufacture_and_distribution', 'packaging_and_delivery', 'price_and_preference', 'customer_service', 'overall_experience'])

for aspect in ['content_related', 'author', 'quality_and_appearance', 'manufacture_and_distribution', 'packaging_and_delivery', 'price_and_preference', 'customer_service', 'overall_experience']:
    print(f'Classification report for aspect: {aspect}\n')
    print(classification_report(df_true[aspect], df_pred[aspect], digits=5))
    print()

Classification report for aspect: content_related

              precision    recall  f1-score   support

           0    0.88170   0.87196   0.87680       453
           1    0.76230   0.77824   0.77019       239

    accuracy                        0.83960       692
   macro avg    0.82200   0.82510   0.82349       692
weighted avg    0.84046   0.83960   0.83998       692


Classification report for aspect: author

              precision    recall  f1-score   support

           0    0.96243   1.00000   0.98085       666
           1    0.00000   0.00000   0.00000        26

    accuracy                        0.96243       692
   macro avg    0.48121   0.50000   0.49043       692
weighted avg    0.92627   0.96243   0.94400       692


Classification report for aspect: quality_and_appearance

              precision    recall  f1-score   support

           0    0.85211   0.78827   0.81895       307
           1    0.84069   0.89091   0.86507       385

    accuracy                 

In [47]:
df_true = df_test.iloc[:, 1:]
df_pred = predict_test

for aspect in ['content_related', 'author', 'quality_and_appearance', 'manufacture_and_distribution', 'packaging_and_delivery', 'price_and_preference', 'customer_service', 'overall_experience']:
    print(f'Classification report for aspect: {aspect}\n')
    print(classification_report(df_true[aspect], df_pred[aspect], digits=5))
    print()

Classification report for aspect: content_related

              precision    recall  f1-score   support

          -1    0.00000   0.00000   0.00000         0
           0    0.87609   0.88962   0.88280       453
           1    0.68020   0.74860   0.71277       179
           2    0.50000   0.42500   0.45946        40
           3    0.00000   0.00000   0.00000        20

    accuracy                        0.80058       692
   macro avg    0.41126   0.41265   0.41101       692
weighted avg    0.77836   0.80058   0.78883       692


Classification report for aspect: author

              precision    recall  f1-score   support

           0    0.96243   1.00000   0.98085       666
           1    0.00000   0.00000   0.00000        19
           2    0.00000   0.00000   0.00000         6
           3    0.00000   0.00000   0.00000         1

    accuracy                        0.96243       692
   macro avg    0.24061   0.25000   0.24521       692
weighted avg    0.92627   0.96243   0

In [48]:
del model

## BiLSTM + BiGRU + Conv1D

In [49]:
with device:

    # Number of labels
    num_label = 8

    # Input layer
    inputs = Input(shape=(MAX_LENGTH,))

    # Embedding layer
    embed = Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        embeddings_initializer=Constant(embedding_matrix),
        trainable=True
    )(inputs)
    
    dropout1 = SpatialDropout1D(0.2)(embed)

    lstm = Bidirectional(LSTM(units = 200, activation = 'tanh', return_sequences = True))(dropout1)
    conv_lstm = Conv1D(128, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(lstm)

    gru = Bidirectional(GRU(units = 200, activation = 'tanh', return_sequences = True))(dropout1)
    conv_gru = Conv1D(128, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(gru)


    avg_pool1 = GlobalAveragePooling1D()(conv_lstm)
    max_pool1 = GlobalMaxPooling1D()(conv_lstm)

    avg_pool2 = GlobalAveragePooling1D()(conv_gru)
    max_pool2 = GlobalMaxPooling1D()(conv_gru)


    concat = Concatenate(axis=-1)([avg_pool1, max_pool1, avg_pool2, max_pool2])
    dense1 = Dense(units = 256, activation = 'relu')(concat)
    dense2 = Dense(units = 128, activation = 'relu')(dense1)
    dropout1 = Dropout(rate = 0.2)(dense2)
    dense3 = Dense(units = 64, activation = 'relu')(dropout1)
    dense4 = Dense(units = 32, activation = 'relu')(dense3)
    
    # Output layers
    out_aspect = Dense(units=num_label, activation='sigmoid', name='out_aspect')(dense4)

    # Helper function to concatenate aspect and dense4 output
    def create_output_layer(aspect_index, name):
        aspect_slice = out_aspect[:, aspect_index:aspect_index + 1]
        concatenated = tf.keras.layers.Concatenate(axis=1)([aspect_slice, dense4])
        output_layer = Dense(units=4, activation='softmax', name=name)(concatenated)
        return output_layer

    out_content_related = create_output_layer(0, 'out_content_related')
    out_author = create_output_layer(1, 'out_author')
    out_quality_and_appearance = create_output_layer(2, 'out_quality_and_appearance')
    out_manufacture_and_distribution = create_output_layer(3, 'out_manufacture_and_distribution')
    out_packaging_and_delivery = create_output_layer(4, 'out_packaging_and_delivery')
    out_price_and_preference = create_output_layer(5, 'out_price_and_preference')
    out_customer_service = create_output_layer(6, 'out_customer_service')
    out_overall_experience = create_output_layer(7, 'out_overall_experience')

    # Define the model
    model = Model(
        inputs=inputs,
        outputs=[
            out_aspect, out_content_related, out_author, out_quality_and_appearance,
            out_manufacture_and_distribution, out_packaging_and_delivery,
            out_price_and_preference, out_customer_service, out_overall_experience
        ]
    )
    
    losses = {
        "out_aspect": "binary_crossentropy",
        "out_content_related": "categorical_crossentropy",
        "out_author": "categorical_crossentropy",
        "out_quality_and_appearance": "categorical_crossentropy",
        "out_manufacture_and_distribution": "categorical_crossentropy",
        "out_packaging_and_delivery": "categorical_crossentropy",
        "out_price_and_preference": "categorical_crossentropy",
        "out_customer_service": "categorical_crossentropy",
        "out_overall_experience": "categorical_crossentropy"
    }
 
    metrics={
        "out_aspect": 'accuracy',
        "out_content_related": 'accuracy',
        "out_author": 'accuracy',
        "out_quality_and_appearance": 'accuracy',
        "out_manufacture_and_distribution": 'accuracy',
        "out_packaging_and_delivery": 'accuracy',
        "out_price_and_preference": 'accuracy',
        "out_customer_service": 'accuracy',
        "out_overall_experience": 'accuracy'
    }
    
    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss=losses,
        metrics=metrics
    )

model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_4         │ (None, 128, 1024) │ 65,537,024 │ input_layer_4[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spatial_dropout1d_2 │ (None, 128, 1024) │          0 │ embedding_4[0][0] │
│ (SpatialDropout1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_4     │ (None, 128, 400)  │  1,960,000 │ spatial_dropout1… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_5     │ (None, 128, 400)  │  1,471,200 │ spatial_dropout1… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 127, 128)  │    102,528 │ bidirectional_4[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 127, 128)  │    102,528 │ bidirectional_5[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 128)       │          0 │ conv1d_2[0][0]    │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ conv1d_2[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 128)       │          0 │ conv1d_3[0][0]    │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ conv1d_3[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_34      │ (None, 512)       │          0 │ global_average_p… │
│ (Concatenate)       │                   │            │ global_max_pooli… │
│                     │                   │            │ global_average_p… │
│                     │                   │            │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_12 (Dense)    │ (None, 256)       │    131,328 │ concatenate_34[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_13 (Dense)    │ (None, 128)       │     32,896 │ dense_12[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 128)       │          0 │ dense_13[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_14 (Dense)    │ (None, 64)        │      8,256 │ dropout_4[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_15 (Dense)    │ (None, 32)        │      2,080 │ dense_14[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ out_aspect (Dense)  │ (None, 8)         │        264 │ dense_15[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_32         │ (None, 1)         │          0 │ out_aspect[0][0]

 Total params: 69,349,192 (264.55 MB)

 Trainable params: 69,349,192 (264.55 MB)

 Non-trainable params: 0 (0.00 B)

In [50]:
# Fit model
history = model.fit(
    x=padded_train,
    y={
        "out_aspect": label_aspect_train,
        "out_content_related": content_related_train,
        "out_author": author_train,
        "out_quality_and_appearance": quality_and_appearance_train,
        "out_manufacture_and_distribution": manufacture_and_distribution_train,
        "out_packaging_and_delivery": packaging_and_delivery_train,
        "out_price_and_preference": price_and_preference_train,
        "out_customer_service": customer_service_train,
        "out_overall_experience": overall_experience_train
    },
    validation_data=(
        padded_val,
        {
            "out_aspect": label_aspect_val,
            "out_content_related": content_related_val,
            "out_author": author_val,
            "out_quality_and_appearance": quality_and_appearance_val,
            "out_manufacture_and_distribution": manufacture_and_distribution_val,
            "out_packaging_and_delivery": packaging_and_delivery_val,
            "out_price_and_preference": price_and_preference_val,
            "out_customer_service": customer_service_val,
            "out_overall_experience": overall_experience_val
        }
    ),
    batch_size=BATCH_SIZE,
    epochs=100,
    callbacks=EarlyStopping(
        monitor='val_loss', 
        patience=3,
        min_delta=0.001,
        mode='min',
        verbose=1
    ),
    verbose=1
)

Epoch 1/100
170/170 ━━━━━━━━━━━━━━━━━━━━ 23s 88ms/step - loss: 8.5512 - out_aspect_accuracy: 0.0076 - out_author_accuracy: 0.7940 - out_content_related_accuracy: 0.4864 - out_customer_service_accuracy: 0.8019 - out_manufacture_and_distribution_accuracy: 0.7756 - out_overall_experience_accuracy: 0.7035 - out_packaging_and_delivery_accuracy: 0.3236 - out_price_and_preference_accuracy: 0.8849 - out_quality_and_appearance_accuracy: 0.3008 - val_loss: 5.5133 - val_out_aspect_accuracy: 0.0101 - val_out_author_accuracy: 0.9523 - val_out_content_related_accuracy: 0.6806 - val_out_customer_service_accuracy: 0.9725 - val_out_manufacture_and_distribution_accuracy: 0.9957 - val_out_overall_experience_accuracy: 0.7355 - val_out_packaging_and_delivery_accuracy: 0.4870 - val_out_price_and_preference_accuracy: 0.9162 - val_out_quality_and_appearance_accuracy: 0.4321
Epoch 2/100
170/170 ━━━━━━━━━━━━━━━━━━━━ 14s 82ms/step - loss: 5.7156 - out_aspect_accuracy: 0.0085 - out_author_accuracy: 0.9561 - out_c

### Evaluation

In [51]:
pred_test = model.predict(padded_test)

22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step


In [52]:
pred_aspect_test = round(pd.DataFrame(pred_test[0]), 0)
aspect_true = label_aspect_test.flatten()
aspect_predict = pred_aspect_test.values.flatten()

weighted_f1_aspect = f1_score(aspect_true, aspect_predict, average='weighted')
print(f'Weighted F1 Score for aspect detection: {weighted_f1_aspect}')

Weighted F1 Score for aspect detection: 0.8874597759407614


In [53]:
pred_content_related_test = np.where(np.argmax(pred_test[1], axis=1)==3, -1, np.argmax(pred_test[1], axis=1))
pred_author_test = np.where(np.argmax(pred_test[2], axis=1)==3, -1, np.argmax(pred_test[2], axis=1))
pred_quality_and_appearance_test = np.where(np.argmax(pred_test[3], axis=1)==3, -1, np.argmax(pred_test[3], axis=1))
pred_manufacture_and_distribution_test = np.where(np.argmax(pred_test[4], axis=1)==3, -1, np.argmax(pred_test[4], axis=1))
pred_packaging_and_delivery_test = np.where(np.argmax(pred_test[5], axis=1)==3, -1, np.argmax(pred_test[5], axis=1))
pred_price_and_preference_test = np.where(np.argmax(pred_test[6], axis=1)==3, -1, np.argmax(pred_test[6], axis=1))
pred_customer_service_test = np.where(np.argmax(pred_test[7], axis=1)==3, -1, np.argmax(pred_test[7], axis=1))
pred_overall_experience_test = np.where(np.argmax(pred_test[8], axis=1)==3, -1, np.argmax(pred_test[8], axis=1))

predict_test = pd.DataFrame({
    'content_related': pred_content_related_test,
    'author': pred_author_test,
    'quality_and_appearance': pred_quality_and_appearance_test,
    'manufacture_and_distribution': pred_manufacture_and_distribution_test,
    'packaging_and_delivery': pred_packaging_and_delivery_test,
    'price_and_preference': pred_price_and_preference_test,
    'customer_service': pred_customer_service_test,
    'overall_experience': pred_overall_experience_test
})

sentiment_true = df_test.iloc[:, 1:].values.flatten()
sentiment_predict = predict_test.values.flatten()

weighted_f1_sentiment = f1_score(sentiment_true, sentiment_predict, average='weighted')
print(f'Weighted F1 Score for sentiment classification: {weighted_f1_sentiment}')

Weighted F1 Score for sentiment classification: 0.8740874757156175


In [54]:
del model